Change file_path to specified folder

In [22]:
import pandas as pd
import csv

# Specify the file paths
file_path = '/Users/ardadinc/Desktop/PRIA/Robinhood/RH_Files/Raw_Robhinhood.csv'
corrected_file_path = '/Users/ardadinc/Desktop/PRIA/Robinhood/RH_Files/Robinhood_Corrected.csv'

# Read and clean the CSV file
with open(file_path, 'r', encoding='utf-8') as infile, open(corrected_file_path, 'w', encoding='utf-8') as outfile:
    temp_line = ""
    for line in infile:
        if 'CUSIP' in line:
            temp_line = temp_line.strip() + " " + line.strip()
        else:
            if temp_line:
                outfile.write(temp_line + '\n')
                temp_line = ""
            temp_line = line.strip()
    if temp_line:
        outfile.write(temp_line + '\n')

# Read the cleaned CSV file with error handling
try:
    df = pd.read_csv(corrected_file_path, delimiter=',', encoding='utf-8', engine='python', quoting=csv.QUOTE_ALL, on_bad_lines='skip')
    print("CSV file read successfully.")
except pd.errors.ParserError as e:
    print(f"ParserError: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

# Display the first few rows to understand the structure
print(df.head())

# Inspect the columns
print(df.columns)

# Assuming the columns might contain leading/trailing spaces, we strip them
df.columns = df.columns.str.strip()

# Convert columns to appropriate data types
if 'Quantity' in df.columns:
    df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')
if 'Price' in df.columns:
    df['Price'] = df['Price'].replace('[\$,]', '', regex=True).replace('', '0').astype(float)
if 'Amount' in df.columns:
    df['Amount'] = df['Amount'].replace('[\$,()]', '', regex=True).replace('', '0').astype(float)

# Filter out rows with quantity less than 1 if 'Quantity' column exists
if 'Quantity' in df.columns:
    df_filtered = df[df['Quantity'] >= 1]
else:
    df_filtered = df

# Filter out rows where 'Trans Code' is not 'BUY' or 'SELL'
df_filtered = df[df['Trans Code'].str.upper().isin(['BUY', 'SELL'])]

# Check if the necessary columns are present before creating the result DataFrame
required_columns = ['Instrument', 'Activity Date', 'Trans Code', 'Quantity', 'Price']
missing_columns = [col for col in required_columns if col not in df_filtered.columns]

if not missing_columns:
    # Create a DataFrame with just the required columns
    result_df = df_filtered[required_columns]

    # Display the DataFrame
    print(result_df)

    # Export the result_df to a new CSV file
    output_file_path = '/Users/ardadinc/Desktop/PRIA/Robinhood/RH_Files/Filtered_Transactions.csv'
    result_df.to_csv(output_file_path, index=False)

    print(f"Filtered transactions exported to {output_file_path}")
else:
    print(f"Missing columns in the data: {missing_columns}")


CSV file read successfully.
  Activity Date Process Date Settle Date Instrument  \
0     6/13/2024    6/13/2024   6/14/2024       STRR   
1     6/13/2024    6/13/2024   6/14/2024       BSFC   
2     6/13/2024    6/13/2024   6/13/2024        VLD   
3     6/12/2024    6/12/2024   6/13/2024       TNXP   
4     6/12/2024    6/12/2024   6/13/2024        ICU   

                              Description Trans Code Quantity  Price   Amount  
0            Star Equity CUSIP: 85513Q103        Buy        1  $0.82  ($0.82)  
1        Blue Star Foods CUSIP: 09606H309       Sell     0.02  $2.40    $0.05  
2                 Velo3D CUSIP: 92259N104        SPR       1S    NaN      NaN  
3  Tonix Pharmaceuticals CUSIP: 890260847       Sell        1  $1.11    $1.11  
4        SeaStar Medical CUSIP: 81256L203       Sell        1  $3.72    $3.72  
Index(['Activity Date', 'Process Date', 'Settle Date', 'Instrument',
       'Description', 'Trans Code', 'Quantity', 'Price', 'Amount'],
      dtype='object')
  

FIX ROBINHOOD CSV FILE TO FORMATE CORRECTLY

In [24]:
import pandas as pd

# Specify the file paths
file_path = '/Users/ardadinc/Desktop/PRIA/Robinhood/RH_Files/Filtered_Transactions.csv'
profit_output_file_path = '/Users/ardadinc/Desktop/PRIA/Robinhood/RH_Files/Robinhood_Profit.csv'
no_match_output_file_path = '/Users/ardadinc/Desktop/PRIA/Robinhood//RH_Files/Robinhood_NoMatches.csv'


# Read the CSV file
df = pd.read_csv(file_path)

# Strip leading/trailing spaces from columns
df.columns = df.columns.str.strip()

# Ensure 'Trans Code' values are uppercase for consistency
df['Trans Code'] = df['Trans Code'].str.upper()

# Filter out rows with quantity less than 1
df = df[df['Quantity'] >= 1]

# Separate the buy and sell transactions
df_buy = df[df['Trans Code'] == 'BUY']
df_sell = df[df['Trans Code'] == 'SELL']

# Initialize lists to store profit calculations and instruments without matching transactions
profit_list = []
no_match_list = []

# Find matching instruments and calculate profit
for instrument in df['Instrument'].unique():
    df_buy_instrument = df_buy[df_buy['Instrument'] == instrument]
    df_sell_instrument = df_sell[df_sell['Instrument'] == instrument]
    
    if not df_buy_instrument.empty and not df_sell_instrument.empty:
        for _, buy_row in df_buy_instrument.iterrows():
            for _, sell_row in df_sell_instrument.iterrows():
                quantity = min(buy_row['Quantity'], sell_row['Quantity'])
                profit = (sell_row['Price'] - buy_row['Price']) * quantity
                profit_list.append({
                    'Instrument': instrument,
                    'Activity Date': buy_row['Activity Date'],
                    'Buy Price': buy_row['Price'],
                    'Sell Price': sell_row['Price'],
                    'Quantity': quantity,
                    'Profit': profit
                })
    else:
        no_match_list.append(instrument)

# Create DataFrames for profit calculations and instruments without matching transactions
profit_df = pd.DataFrame(profit_list)
no_match_df = pd.DataFrame(no_match_list, columns=['Instrument'])

# Calculate the total profit
total_profit = profit_df['Profit'].sum()

# Print the total profit formatted to two decimal places
print(f"Total Profit: {total_profit:.2f}")

# Export the DataFrames to CSV files
profit_df.to_csv(profit_output_file_path, index=False)
no_match_df.to_csv(no_match_output_file_path, index=False)

print(f"Profit calculations exported to {profit_output_file_path}")
print(f"Instruments without matching transactions exported to {no_match_output_file_path}")

# Display the first few rows of the profit_df to understand the structure
print(profit_df.head())


Total Profit: 786.15
Profit calculations exported to /Users/ardadinc/Desktop/PRIA/Robinhood/RH_Files/Robinhood_Profit.csv
Instruments without matching transactions exported to /Users/ardadinc/Desktop/PRIA/Robinhood//RH_Files/Robinhood_NoMatches.csv
  Instrument Activity Date  Buy Price  Sell Price  Quantity  Profit
0       TNXP      6/6/2024       0.11        1.11       1.0    1.00
1       TNXP      6/6/2024       0.11        1.80       1.0    1.69
2       TNXP      5/9/2023       0.38        1.11       1.0    0.73
3       TNXP      5/9/2023       0.38        1.80       1.0    1.42
4        ICU      6/6/2024       0.21        3.72       1.0    3.51


In [27]:
import pandas as pd

# Specify the file paths
input_file_path = '/Users/ardadinc/Desktop/PRIA/Robinhood/RH_Files/Robinhood_Profit.csv'
output_file_path = '/Users/ardadinc/Desktop/PRIA/Robinhood/RH_Files/Robinhood_Final.csv'

# Read the CSV file
df = pd.read_csv(input_file_path)

# Strip leading/trailing spaces from columns
df.columns = df.columns.str.strip()

# Display the columns to ensure they are correctly named
print("Columns in DataFrame:", df.columns)

# Check if the necessary columns are present
required_columns = ['Instrument', 'Activity Date', 'Profit']
missing_columns = [col for col in required_columns if col not in df.columns]

if missing_columns:
    print(f"Missing columns in the data: {missing_columns}")
else:
    # Check for duplicates and filter based on activity date and profit
    def filter_duplicates(df):
        df_filtered = df.copy()
        # Sort by Instrument, Activity Date, and Profit
        df_filtered = df_filtered.sort_values(by=['Instrument', 'Activity Date', 'Profit'], ascending=[True, True, False])
        # Drop duplicates keeping the one with the larger profit for the same activity date
        df_filtered = df_filtered.drop_duplicates(subset=['Instrument', 'Activity Date'], keep='first')
        return df_filtered

    filtered_df = filter_duplicates(df)

    # Export the filtered DataFrame to a new CSV file
    filtered_df.to_csv(output_file_path, index=False)

    print(f"Filtered profit data exported to {output_file_path}")

    # Display the first few rows of the filtered DataFrame to understand the structure
    print(filtered_df.head())


Columns in DataFrame: Index(['Instrument', 'Activity Date', 'Buy Price', 'Sell Price', 'Quantity',
       'Profit'],
      dtype='object')
Filtered profit data exported to /Users/ardadinc/Desktop/PRIA/Robinhood/RH_Files/Robinhood_Final.csv
    Instrument Activity Date  Buy Price  Sell Price  Quantity  Profit
102       ACON      1/3/2024       0.20        3.27       1.0    3.07
166       ADTX     8/17/2023       0.40       11.44       1.0   11.04
152       AEHL     9/15/2023       0.33        2.71       1.0    2.38
174       AGFY      7/3/2023       0.18        2.77       1.0    2.59
147       AGRI    10/10/2023       0.08        2.93       1.0    2.85
